In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 2 13:20:39 2022
Updated on Mon Jun 20 20:11:48 2022

@author: Binyang
"""

import requests
from requests import get
from bs4 import BeautifulSoup
import time
from pathlib import Path
import json
import sys
from importlib import reload
import pandas as pd

In [18]:
def sleep(duration):
    print(f'sleep {duration}s')
    time.sleep(duration)

def parse(text, parser='html.parser'):
    return BeautifulSoup(text, parser)

def fetch(url, file_path=None, session=None, max_attempts=10, custom_headers=None):
    if custom_headers is None:
        custom_headers = {}

    if file_path is not None:
        print(f'Fetch {url} to {file_path}')
        if Path(file_path).exists():
            print('Already exists. Skipped.')
            return None
    else:
        print(f'Fetch {url}')

    for attempt in range(max_attempts):
        if session is not None:
            response = session.get(url, headers=custom_headers)
        else:
            response = get(url, headers=custom_headers)

        # handle HTTP 429 Too Many Requests
        if response.status_code == 429:
            delay = (attempt + 1) * 10
            print(f'Retrying in {delay} seconds')
            time.sleep(delay)
        else:
            break

    response.raise_for_status()

    if session is not None:
        print(f'Session cookies: {len(session.cookies)}')

    if file_path is not None:
        with open(file_path, 'wb') as file:
            file.write(response.content)

    return response.text

In [19]:
##test
custom_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

doc = parse(fetch("https://asmedigitalcollection.asme.org/mechanicaldesign/issue/browse-by-year", custom_headers=custom_headers))

Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/issue/browse-by-year


In [257]:
all_docs_info = []
main_url = "https://asmedigitalcollection.asme.org"
columns_to_take = ["title", "abstract", "doi", "year", "issue", "citation_info", "authors", "pdf_link", "issue_section",
                   "topics", "keywords", "related article", "source of related articles", "cited times", 
                   "author affiliation", "reference"]
doc = parse(fetch("https://asmedigitalcollection.asme.org/mechanicaldesign/issue/browse-by-year", custom_headers=custom_headers))
##get the links to the webpage of each year
for year_link in doc.select("div.widget-ContentBrowseAllYearsManifest a"):
    year = int(year_link.text.strip())

    ##scrape JMD website by year
    if(year == 2022):
        year_href = year_link["href"]
        year_doc = parse(fetch(year_href, custom_headers=custom_headers))
        
        ##scrape all issues in a certain year
        for issue_link in year_doc.select("div.widget-ContentBrowseByYearManifest a"):
            issue_text = issue_link.text.strip()
            issue_link  = main_url + issue_link["href"]
            print(year, issue_link)
            issue_doc = parse(fetch(issue_link, custom_headers=custom_headers))
            
            ##get the link to each paper in a certain issue
            ##scrape the desired data from paper webpage
            for article in issue_doc.select("div.al-article-item-wrap"):
                article_obj = {}
                for column in columns_to_take:
                    article_obj[column] = ""
                article_obj["year"] = year
                article_obj["issue"] = issue_text
                ##title
                if article.select("h5.item-title"):
                    article_obj["title"] = article.select("h5.item-title")[0].text.strip()
                ##author
                article_obj["authors"] = ";".join([author.text.strip() for author in article.select("div.al-authors-list")])
                ##pdf link
                if article.select("a.article-pdfLink"):
                    article_obj["pdf_link"] = main_url + article.select("a.article-pdfLink")[0]["href"]
                ##other info
                if article.select("h5.item-title a"):
                    article_link = main_url + article.select("h5.item-title a")[0]["href"]
                    article_doc = parse(fetch(article_link, custom_headers=custom_headers))
                    ##abstract
                    article_obj["abstract"] = "\n".join([abstract_section.text for abstract_section in article_doc.select(".abstract")])
                    ##cite_info
                    article_obj["citation_info"] = ";".join([section.text.strip() for section in article_doc.select(".ww-citation-primary")])
                    ##doi
                    if article_doc.select("a.ww-doi-link"):
                        article_obj["doi"] = article_doc.select("a.ww-doi-link")[0]["href"]
                    ##issue_section
                    article_obj["issue_section"] = " ".join([phrase.text for phrase in article_doc.select(".content-metadata-tocSections")]).strip()
                    ##topic
                    article_obj["topics"] = " ".join([phrase.text for phrase in article_doc.select(".content-metadata-topics")]).strip()
                    ##keywords
                    article_obj["keywords"] = " ".join([phrase.text for phrase in article_doc.select(".content-metadata-keywords")]).strip()
                    ##most related ASME papers assigned by ASME 
                    article_obj["related article"] = ";".join([related_articles.text for related_articles in article_doc.select("div.article-title a")]).strip()
                    article_obj["source of related articles"] = ";".join([sources.text for sources in article_doc.select("div.content-meta")]).strip()
                    ##times being cited
                    article_obj["cited times"] = ";".join([web.text for web in article_doc.select("div.article-cited-link-wrap a")]).strip()
                    ##author affiliations
                    aff0 = ''
                    for affiliation in article_doc.select("div.author-affiliation"):
                        for aff in affiliation.select("div.aff"):
                            aff0 = ";".join([aff0,aff.text])
                    article_obj["author affiliation"] = aff0.strip()[1:]
                    ##references of a focal paper
                    if article_doc.select("div.ref-list"):
                        reference = ''
                        doix = ''
                        sourcex = ''
                        titlex = ''
                        for ref in article_doc.select("div.ref-list")[0].select('div.citation'):
                            if ref.select('div.article-title'):
                                titlex = "title::" + ref.select('div.article-title')[0].text.strip()
                        #     print (ref.text.strip())
                            if ref.select('div.source'):
                                sourcex = "source::" + ref.select('div.source')[0].text.strip()
                            if ref.select('div.crossref-doi a'):
                                doix = "doi::" + ref.select('div.crossref-doi a')[0]['href'].replace('dx.', '')
                            referencex = ';'.join([titlex, sourcex, doix])
                            reference = '||'.join([reference, referencex])
                        article_obj["reference"] = reference[2:]
                print(article_obj["title"])
                all_docs_info.append(article_obj)

print("Number of docs: ", len(all_docs_info))

df_with_articles = pd.DataFrame(all_docs_info)

df_with_articles.to_excel("jmd_all_updated.xlsx", encoding="utf-8")

Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/issue/browse-by-year
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/issue/browse-by-year/2022
2022 https://asmedigitalcollection.asme.org/mechanicaldesign/issue/144/1
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/issue/144/1
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/1/011401/1114346/Factors-Impacting-Highly-Innovative-Designs-Idea
Factors Impacting Highly Innovative Designs: Idea Fluency, Timing, and Order
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/1/011402/1114944/Toward-a-Theory-of-Systems-Engineering
Toward a Theory of Systems Engineering
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/1/011701/1114343/An-Optimization-Framework-for-the-Design-of-Cable
An Optimization Framework for the Design of Cable Harness Layouts in Planar Interconnected Systems
Fetch https://asmedigitalcollection.asme.org/mechanicald

Mission Engineering and Design Using Real-Time Strategy Games: An Explainable AI Approach
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/2/021711/1121902/GANTL-Toward-Practical-and-Real-Time-Topology
GANTL: Toward Practical and Real-Time Topology Optimization With Conditional Generative Adversarial Networks and Transfer Learning
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/2/021712/1122916/Inverse-Design-of-Two-Dimensional-Airfoils-Using
Inverse Design of Two-Dimensional Airfoils Using Conditional Generative Models and Surrogate Log-Likelihoods
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/2/021713/1122919/Design-as-a-Marked-Point-Process
Design as a Marked Point Process
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/2/022001/1122891/An-Image-Driven-Uncertainty-Inverse-Method-for
An Image-Driven Uncertainty Inverse Method for Sheet Metal Forming Problems
Fetch https://asmedigi

A Large Range Compliant Nano-Manipulator Supporting Electron Beam Lithography
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/4/045001/1120820/Corrugated-Diaphragm-Actuator-for-Soft-Robotic
Corrugated Diaphragm Actuator for Soft Robotic Applications
2022 https://asmedigitalcollection.asme.org/mechanicaldesign/issue/144/5
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/issue/144/5
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/5/051401/1120718/Can-Induced-Gratitude-Improve-Creative-Performance
Can Induced Gratitude Improve Creative Performance on Repurposing Tasks?
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/5/051402/1121228/A-Graph-Partitioning-Technique-to-Optimize-the
A Graph Partitioning Technique to Optimize the Physical Integration of Functional Requirements for Axiomatic Design
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/5/051701/1123754/Stochastic-Crashw

Level Set-Based Structural Optimization With Uniform Wall Thickness for Wire-Fed Metal Additive Manufacturing
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/7/071703/1131073/Deep-Generative-Tread-Pattern-Design-Framework-for
Deep Generative Tread Pattern Design Framework for Efficient Conceptual Design
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/7/071705/1135468/ActivationNet-Representation-Learning-to-Predict
ActivationNet: Representation Learning to Predict Contact Quality of Interacting 3D Surfaces in Engineering Designs
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/7/072001/1135469/Design-for-Inspectability-A-Framework-to-Increase
Design for Inspectability: A Framework to Increase Inspectability of Additive Manufacturing Parts for Pulse-Echo Ultrasonic Inspection Methods
Fetch https://asmedigitalcollection.asme.org/mechanicaldesign/article/144/7/073301/1131323/Configuration-Design-and-Screening-of-

In [16]:
# import urllib
# def download_pdf_by_link(download_url, filename):
#     try:
#         response = urllib.request.urlopen(download_url)
#         pdf_data = response.read()
#         assert len(pdf_data) > 5000
#         file = open(filename, 'wb')
#         file.write(pdf_data)
#         file.close()
#         print("Downloaded file ", filename)
#     except Exception as err:
#         print(err)

In [54]:
custom_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

In [5]:
def download_pdf_by_link(download_url, filename):
    try:
        r = requests.get(download_url, headers = custom_headers)
        with open(filename, 'wb') as f:
            f.write(r.content)
        print("Downloaded file ", filename)
    except Exception as err:
        print(err)

In [13]:
import os
import time
# os.makedirs("folder_with_pdfs", exist_ok=True)

df_with_articles = pd.read_excel('jmd_all_updated.xlsx')
n = 0
for i in range(0,len(df_with_articles)):
    print(df_with_articles["pdf_link"].values[i])
    if df_with_articles["pdf_link"].values[i].strip():
        download_pdf_by_link(df_with_articles["pdf_link"].values[i], os.path.join("folder_with_pdfs", "%d.pdf"%i))
        n += 1
        if n%15 == 0:
            time.sleep(5)

https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001000/5806922/031003_1.pdf
Downloaded file  folder_with_pdfs\3847.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001202/5808352/031004_1.pdf
Downloaded file  folder_with_pdfs\3848.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001108/5808531/031005_1.pdf
Downloaded file  folder_with_pdfs\3849.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001211/5807457/031006_1.pdf
Downloaded file  folder_with_pdfs\3850.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001213/5807943/031007_1.pdf
Downloaded file  folder_with_pdfs\3851.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001214/6729516/031008_1.pdf
Downloaded file  folder_with_pdfs\3852.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1

Downloaded file  folder_with_pdfs\3900.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001531/5807857/071003_1.pdf
Downloaded file  folder_with_pdfs\3901.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001722/5807505/071004_1.pdf
Downloaded file  folder_with_pdfs\3902.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001667/5807628/071005_1.pdf
Downloaded file  folder_with_pdfs\3903.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001668/5807916/071006_1.pdf
Downloaded file  folder_with_pdfs\3904.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001525/5807737/071007_1.pdf
Downloaded file  folder_with_pdfs\3905.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001873/5807381/071008_1.pdf
Downloaded file  folder_with_pdfs\3906.pdf
https://asmedigitalcollection.asm

Downloaded file  folder_with_pdfs\3954.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002560/5521625/111002_1.pdf
Downloaded file  folder_with_pdfs\3955.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002663/5522778/111003_1.pdf
Downloaded file  folder_with_pdfs\3956.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002655/5522751/111004_1.pdf
Downloaded file  folder_with_pdfs\3957.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002202/5522498/111005_1.pdf
Downloaded file  folder_with_pdfs\3958.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002763/5521801/111006_1.pdf
Downloaded file  folder_with_pdfs\3959.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4001408/5521956/111007_1.pdf
Downloaded file  folder_with_pdfs\3960.pdf
https://asmedigitalcollection.asm

Downloaded file  folder_with_pdfs\4008.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4003704/5496028/030201_1.pdf
Downloaded file  folder_with_pdfs\4009.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4003567/5496167/030301_1.pdf
Downloaded file  folder_with_pdfs\4010.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4003499/6730981/031001_1.pdf
Downloaded file  folder_with_pdfs\4011.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4003412/5496874/031002_1.pdf
Downloaded file  folder_with_pdfs\4012.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4002978/5497212/031003_1.pdf
Downloaded file  folder_with_pdfs\4013.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4003537/5497033/031004_1.pdf
Downloaded file  folder_with_pdfs\4014.pdf
https://asmedigitalcollection.asm

Downloaded file  folder_with_pdfs\4062.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004117/5925928/071002_1.pdf
Downloaded file  folder_with_pdfs\4063.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004198/5926575/071003_1.pdf
Downloaded file  folder_with_pdfs\4064.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004222/5925839/071004_1.pdf
Downloaded file  folder_with_pdfs\4065.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004223/5926232/071005_1.pdf
Downloaded file  folder_with_pdfs\4066.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004224/5926474/071006_1.pdf
Downloaded file  folder_with_pdfs\4067.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4004233/5925567/071007_1.pdf
Downloaded file  folder_with_pdfs\4068.pdf
https://asmedigitalcollection.asm

...

Downloaded file  folder_with_pdfs\5350.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045385/6649219/md_142_3_031115.pdf
Downloaded file  folder_with_pdfs\5351.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045602/6649204/md_142_3_031116.pdf
Downloaded file  folder_with_pdfs\5352.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045968/6649129/md_142_3_031117.pdf
Downloaded file  folder_with_pdfs\5353.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045600/6649191/md_142_3_031118.pdf
Downloaded file  folder_with_pdfs\5354.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045176/6649280/md_142_3_031119.pdf
Downloaded file  folder_with_pdfs\5355.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4044691/6649233/md_142_3_031120.pdf
Downloaded file  folder_with_pdfs\

Downloaded file  folder_with_pdfs\5401.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045300/6540446/md_142_7_072003.pdf
Downloaded file  folder_with_pdfs\5402.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045723/6540545/md_142_7_073301.pdf
Downloaded file  folder_with_pdfs\5403.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046073/6540478/md_142_7_073302.pdf
Downloaded file  folder_with_pdfs\5404.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045508/6540527/md_142_7_073501.pdf
Downloaded file  folder_with_pdfs\5405.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046067/6540500/md_142_7_074501.pdf
Downloaded file  folder_with_pdfs\5406.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4045388/6540633/md_142_7_075001.pdf
Downloaded file  folder_with_pdfs\

Downloaded file  folder_with_pdfs\5452.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046809/6536232/md_142_11_111702.pdf
Downloaded file  folder_with_pdfs\5453.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046914/6536843/md_142_11_111703.pdf
Downloaded file  folder_with_pdfs\5454.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046916/6536907/md_142_11_111704.pdf
Downloaded file  folder_with_pdfs\5455.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4047152/6543247/md_142_11_111705.pdf
Downloaded file  folder_with_pdfs\5456.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4047002/6543548/md_142_11_111706.pdf
Downloaded file  folder_with_pdfs\5457.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4046810/6536200/md_142_11_113301.pdf
Downloaded file  folder_with

Downloaded file  folder_with_pdfs\5503.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052192/6756044/md_144_2_021702.pdf
Downloaded file  folder_with_pdfs\5504.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052221/6756166/md_144_2_021703.pdf
Downloaded file  folder_with_pdfs\5505.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052298/6760210/md_144_2_021704.pdf
Downloaded file  folder_with_pdfs\5506.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052299/6760222/md_144_2_021705.pdf
Downloaded file  folder_with_pdfs\5507.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052301/6760373/md_144_2_021706.pdf
Downloaded file  folder_with_pdfs\5508.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052300/6761504/md_144_2_021707.pdf
Downloaded file  folder_with_pdfs\

Downloaded file  folder_with_pdfs\5554.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052906/6808341/md_144_5_051705.pdf
Downloaded file  folder_with_pdfs\5555.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052758/6807469/md_144_5_052001.pdf
Downloaded file  folder_with_pdfs\5556.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052842/6806596/md_144_5_053301.pdf
Downloaded file  folder_with_pdfs\5557.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4053043/6806447/md_144_5_053302.pdf
Downloaded file  folder_with_pdfs\5558.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052904/6806648/md_144_5_054501.pdf
Downloaded file  folder_with_pdfs\5559.pdf
https://asmedigitalcollection.asme.org/mechanicaldesign/article-pdf/doi/10.1115/1.4052970/6806371/md_144_5_054502.pdf
Downloaded file  folder_with_pdfs\

In [203]:
##detect cases that multiple papers share the same pdf file 
for i in range(1,len(df_with_articles)):
# for i in range(1133,1134):
    size = os.path.getsize(os.path.join("folder_with_pdfs", "%d.pdf"%i)) 
    size1 = os.path.getsize(os.path.join("folder_with_pdfs", "%d.pdf"%(i-1))) 
    if size == size1:
        print (i)